In [37]:
#imports
import pandas as pd
import base64
import math
import re
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from nltk.stem.porter import PorterStemmer




In [38]:
#global
p_stemmer = PorterStemmer()
stop_words = stopwords.words('english')
print stop_words

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [ ]:
posts = pd.read_csv('posts.sample.csv').dropna()
posts['Body'] = posts['Body'].apply(lambda x : BeautifulSoup(base64.b64decode(x)).get_text())
posts['Title'] = posts['Title'].apply(lambda x : BeautifulSoup(base64.b64decode(x)).get_text())
posts.head(2)

In [ ]:
comments = pd.read_csv('comments.sample.csv').dropna()
comments['Text'] = comments['Text'].apply(lambda x : BeautifulSoup(base64.b64decode(x)).get_text())
comments.head(2)

In [ ]:
posthistory = pd.read_csv('posthistory.sample.csv').dropna()
posthistory['Text'] = posthistory['Text'].apply(lambda x : BeautifulSoup(base64.b64decode(x)).get_text())
posthistory.head(2)

In [ ]:
users = pd.read_csv('users.sample.csv').dropna()
users['AboutMe'] = users['AboutMe'].apply(lambda x : BeautifulSoup(base64.b64decode(x)).get_text())
users['Location'] = users['Location'].apply(lambda x : BeautifulSoup(base64.b64decode(x)).get_text())
users.head(2)

In [43]:
class Sentences():
    def __init__(self,df,field):
        self.field = field
        self.df = df
    
    def __iter__(self):
      for index, row in self.df.iterrows():
         raw_sentence = row[self.field].replace('\n','').lower()
         raw_tokens = filter(None, re.split("[ ?]+",raw_sentence))
         stem_tokens = [p_stemmer.stem(tok) for tok in raw_tokens]
         tokens = [tok for tok in stem_tokens if not tok in stop_words]       
         yield tokens
        

allposts = Sentences(posts,'Body')


In [46]:
#word2vec
#model = gensim.models.Word2Vec(psen)

dictionary = corpora.Dictionary(allposts)
corpus = [dictionary.doc2bow(text) for text in allposts]

print(corpus[0])

[(0, 2), (1, 1), (2, 1), (3, 1)]


In [47]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary, passes=20)

In [48]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

[(4, u'0.030*would + 0.030*student + 0.030*say'), (21, u'0.029*one + 0.029*inning + 0.019*noun'), (27, u'0.084*use + 0.043*differ + 0.043*know')]


In [ ]:
#describe
posts.describe()
posthistory.describe()
comments.describe()
users.describe()